In [ ]:
import pandas as pd
import numpy as np

In [ ]:
class performance_check_plant_id:
    def __init__(self,filedir,filename,pred_time,check_start_date,check_end_date):
        self.filedir = filedir
        self.filename = filename
        self.pred_time = pred_time
        self.check_start_date = check_start_date
        self.check_end_date = check_end_date
        
    def read_result_file(self):
        result_df = pd.read_csv(self.filedir+self.filename+self.pred_time+'.csv',index_col=0)
        result_df = result_df[(result_df.base_date>=self.check_start_date)&(result_df.base_date<=self.check_end_date)].reset_index(drop=True)
        result_df['gen_hour'] = result_df['pv_amount'] / result_df['kpx_capacity']
        result_df['gen_hour_pred'] = result_df['pv_amount_prediction'] / result_df['kpx_capacity']
        return result_df
    
    @staticmethod
    def get_nmae(result_merge):
        tot_nmae = np.mean(result_merge.nmae[result_merge.use>=0.1])
        tc= np.sum(result_merge.pv_amount[result_merge.use>=0.1])*4
        c4 =np.sum(result_merge.pv_amount[(result_merge.use>=0.1)&(result_merge.nmae<=0.06)])*4
        c3 =np.sum(result_merge.pv_amount[(result_merge.use>=0.1)&(result_merge.nmae>0.06)&(result_merge.nmae<=0.08)])*3
        tcc = (c4+c3)/tc
        return tot_nmae
    
    @staticmethod
    def monthly_result_report(monthly_result_df):
        monthly_plant_result = []
        for base_date in monthly_result_df.base_date.unique():
            fslc003_pred0_by_date = monthly_result_df[monthly_result_df.base_date==base_date].reset_index(drop=True)
            for plant_id in fslc003_pred0_by_date.plant_id.unique():
                fslc003_pred0_by_date_pid = fslc003_pred0_by_date[fslc003_pred0_by_date.plant_id==plant_id].reset_index(drop=True)
                monthly_plant_result.append([base_date,plant_id,np.round(performance_check_plant_id.get_nmae(fslc003_pred0_by_date_pid),4)])
        monthly_plant_result_df = pd.DataFrame(monthly_plant_result,columns=['base_date','plant_id','nmae'])
        monthly_plant_result_df_nmae_count = monthly_plant_result_df.pivot_table(index=['plant_id'],values='nmae',aggfunc= lambda x:(x>=0.1).sum()).reset_index()
        monthly_plant_result_df_nmae_count = monthly_plant_result_df_nmae_count.sort_values(by=['nmae']).reset_index(drop=True)
        monthly_plant_result_df_nmae_count = monthly_plant_result_df_nmae_count.rename(columns={'nmae':'nmae_count'})
        return monthly_plant_result_df_nmae_count

In [ ]:
performance_check_plant_id = performance_check_plant_id(
    filedir = "./fSLC003_연구/결과파일/",
    filename = "result_ldaps_",
    pred_time='1',
    check_start_date = "2022-11-01",
    check_end_date = "2022-11-30",
)

In [ ]:
result_df = performance_check_plant_id.read_result_file()
performance_check_plant_id.monthly_result_report(result_df)